In [1]:
from bs4 import BeautifulSoup
import re
import urllib
import pandas as pd
from urllib.request import urlopen

In [2]:
def make_soup(url):
    html = urlopen(url).read()
    return BeautifulSoup(html,'lxml')

In [3]:
def page_scrape(soup):
    review_list = soup.find_all('li',class_='review_item clearfix')
    for review in review_list:
        review_item = review.find('div', class_='review_item_review')
        if review_item is not None:
            title = review_item.find('span',itemprop = 'name').text.strip()
            score = float(review_item.find('span',class_= 'review-score-badge').text.strip())

            review_neg = review_item.find('p',class_='review_neg')
            if review_neg is not None:
                review_neg = review_neg.text.strip()
            review_pos = review_item.find('p',class_='review_pos')
            if review_pos is not None:
                review_pos = review_pos.text.strip()

            stay_date = review_item.find('p',class_='review_staydate')
            if stay_date is not None:
                stay_date = stay_date.text.strip().replace('Stayed in ','')

        review_date = review.find('p',class_='review_item_date').text.strip().replace('Reviewed: ','')

        reviewer = review.find('div',class_='review_item_reviewer')
        if reviewer is not None:
            reviewer_name = reviewer.find('p',class_='reviewer_name')
            if reviewer_name is not None:
                reviewer_name = reviewer_name.find('span',itemprop='name').text.strip()

            reviewer_country = reviewer.find('', class_='reviewer_country')
            if reviewer_country is not None:
                reviewer_country = reviewer_country.find('span',itemprop='name').text.strip()

        review_gen = [review_date,title,score,review_pos,review_neg,reviewer_name,reviewer_country,stay_date]
        review_data.append(review_gen)

In [4]:
num_pages = 81
review_data = []
for i in range(num_pages):
    url_link = 'https://www.booking.com/reviews/sg/hotel/marina-bay-sands.en-gb.html?aid=357004;label=gog235jc-1DCA0oyQE4qAZIM1gDaMkBiAEBmAEJuAEXyAEU2AED6AEBiAIBqAIDuAKR1f3xBcACAQ;sid=8a6dd48ab3f6c651d2176995dfe97040;customer_type=total&hp_nav=0&keep_landing=1&order=featuredreviews&page='+str(i+1)+'&r_lang=en&rows=75&'
    soup = make_soup(url_link)
    page_scrape(soup)

review_data = pd.DataFrame(review_data,columns = ['review_date','title','score','pos','cons','reviewer','review_country','stay_date'])

In [5]:
review_data.to_csv('MBS_data.csv')